## Config

In [ ]:
### Configuration
# Dependencies
import os
import pandas as pd
import numpy as np
from summerplaylist import *

# Environment variables
CLIENTID = os.environ.get('CLIENTID')
CLIENTSECRET = os.environ.get('CLIENTSECRET')
REDIRECT_URI = 'https://example.com/callback'

# Filepaths
artistsKnownPath = os.path.join('data', 'artistsKnown.csv')

# Read in known artists
artistsKnown = pd.read_csv(artistsKnownPath)

# Set max age for new artist songs
maxAge = 5

## Log In

In [43]:
# Authenticate to Spotify
credentials = authenticateToSpotify(CLIENTID, CLIENTSECRET, REDIRECT_URI)

Opening browser for Spotify login...


## Collect new artists

In [16]:
# Collect names of new artists from Music Festival Wizard
artistsNew = fetchMusicFestivalWizard()
# Filter out known artists who we have already listened to
artistsNew = artistsNew.loc[~artistsNew['artistName'].isin(artistsKnown['artistName'])]
artistsNew = artistsNew.head()
# Search spotify for artistIDs
artistsNew['artistID'] = artistsNew.apply(findArtistIDs, credentials=credentials, axis=1)
# Filter out if artistID was known
artistsNew = artistsNew.loc[~artistsNew['artistID'].isin(artistsKnown['artistID'])]
artistsNew

,artistName,artistID
33,AME,2aQnC3DbZB9GbauvhAw7ve


In [44]:
# Print the playlist IDs so user can select an id to add to "New Artists to Explore"
import time
from datetime import datetime
from dateutil.relativedelta import relativedelta
from collections import Counter
import pandas as pd
import numpy as np
from requests_html import HTMLSession
import tekore as tk

spotify, userID, user_token = credentials
with spotify.token_as(user_token):
    # Check if playlist already exists; if not create a new one
    userPlaylists = pd.DataFrame([{'id':x.id, 'name':x.name} for x in spotify.playlists(userID, limit=40).items])

userPlaylists

,id,name
0,3hucYsCUS3jPd5V2bIusA8,Bonnaroo 2024 Official Playlist
1,1zwX8cu5LzyR3TsHQkGVAT,The Governors Ball Music Festival 2024
2,5dzr9wjX23Tbtr5a4w80aH,BOTTLEROCK Festival 2024 Complete Setlist Play...
3,0fAAvQ3fVmaxHisyWqffIq,Just Like Heaven 2024 Official Playlist
4,3zviNq7uibf4AEs2TllQAj,Hangout Music Fest - Official Playlist
5,4rz2SAdaO69hAgtBltyweT,Cruel World 2024 Lineup
6,6MQO9TuoxOnpdcaglTi7LW,Wonderfront 2024
7,3ixT59DDneqNszbtKnJbJ5,Official Kilby Block Party 2024
8,5lJiaKKgCktH0ZpuZPKurN,2024 Beach Life Festival Bands
9,0GNzkoigT71tlRUCWHQsBV,Shaky Knees 2024


In [45]:
# Collect artists from an existing favorited playlist
playlistToFetch = '1zwX8cu5LzyR3TsHQkGVAT' #getPlaylistID(credentials, 'New Artists to Explore')
artistsNew = fetchTracksFromPlaylist(credentials, playlistToFetch)
artistsNew = artistsNew[['artistName', 'artistID']].drop_duplicates('artistID', ignore_index=True).reset_index(drop=True)
# Filter out if artistID was known
artistsNew = artistsNew.loc[~artistsNew['artistID'].isin(artistsKnown['artistID'])]

## Process new artists

In [46]:
artistsNew


,artistName,artistID
1,Swae Lee,1zNqQNIdeOUZHb8zbZRFMX
2,Quavo,0VRj0yCOv2FXJNP47XQnx5
4,Chencho Corleone,37230BxxYs9ksS7OkZw3IU
7,Lyanno,1Ts9of7VPZElwPQnqnDSfW
8,Brray,1GKIlPFdcewHtpDVCQ8zmJ
11,Zendaya,6sCbFbEjbYepqswM1vWjjs
13,Pedro Capó,4QVBYiagIaa6ZGSPMbybpy
15,Arcángel,4SsVbpTthjScTS7U2hmr1X
16,Ñengo Flow,12vb80Km0Ew53ABfJOepVz
17,DJ Luian,64aJYyrXljOodnUG6jvhRD


## Get top songs

In [47]:
# Get their top three songs in last 5 years
songURIs = fetchTopSongs(credentials, artistsNew['artistID'], maxAge=maxAge)
# How many songs are in this round?
print(f"There are {len(songURIs)} songs about to be added")

There are 157 songs about to be added


## Put songs in playlist

In [40]:
# Warning: This will make a new playlist
playlistID = getPlaylistID(credentials, 'New Artists to Explore')
# Add songs to the playlist
addToPlaylist(credentials, songURIs, playlistID)

## Update the list of known artists

In [41]:
# Append artistsNew to artistsKnown.csv 
allArtists = pd.concat([artistsKnown, artistsNew], axis=0, ignore_index=True)
allArtists.to_csv(artistsKnownPath, index=False)

## Create playlist to drop good songs into

In [21]:
# Cull exploreArtists into ArtistHasPotential playlist
winnersPlaylist = getPlaylistID(credentials, 'New Artists to Explore Further')
print(f'Now drop songs you like into this playlist: {winnersPlaylist}')

Now drop songs you like into this playlist: 0Yj1Gqn15HdcDNcV5XJqVm
